# Tutorial: Downloading, Visualizing, and Interpreting GPM IMERG Data

IMERG data (Integrated Multi-satellite Retrievals for GPM) come from the GPM (Global Precipitation Measurement) satellite.

Key characteristics:
- Each data point represents the amount of rainfall in millimeters.
- The data is global, allowing rainfall studies across different countries and regions.
- IMERG combines measurements from multiple satellites to provide reliable and detailed data.

Important: To use IMERG data, you need to create an Earthdata account. This is free and can be done at: https://urs.earthdata.nasa.gov/users/new
The username and password you create can then be added to the .devcontainer file in this folder.

# Step 1: Import the Required Python Libraries

In [ ]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import io
import requests
from netrc import netrc
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from datetime import datetime, timedelta
from tqdm import tqdm
import matplotlib.dates as mdates
from matplotlib.colors import BoundaryNorm, ListedColormap
from IPython.display import display
from tqdm import tqdm
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d
import ipywidgets as widgets
from ipyleaflet import Map, DrawControl
from ipyleaflet import Map, DrawControl, Rectangle
from matplotlib.ticker import AutoMinorLocator
from IPython.display import HTML
from matplotlib.animation import FuncAnimation
import h5py, requests, io
from netrc import netrc
from datetime import datetime, timedelta
import os
from pathlib import Path

# Step 2: Select an area of interest using an interactive map

In [ ]:
def interactive_bbox_ipy(center=(5,5), zoom=5):
    bbox_output = widgets.Output()
    m = Map(center=center, zoom=zoom, layout=widgets.Layout(width='70%', height='500px'))

    # Tool to draw a rectangle on the map
    draw = DrawControl(
        rectangle={"shapeOptions": {"color": "#0000FF"}},
        polygon={}, circle={}, polyline={}, marker={}
    )

    current_rectangle = None
    # Mutable object to store bounding box (BBox)
    bbox_coords = {"lat_min": None, "lat_max": None, "lon_min": None, "lon_max": None}

    def handle_draw(self, action, geo_json):
        nonlocal current_rectangle
        coords = geo_json['geometry']['coordinates'][0]
        lons = [c[0] for c in coords]
        lats = [c[1] for c in coords]

        # Update dictionary with new coordinates
        bbox_coords["lon_min"] = min(lons)
        bbox_coords["lon_max"] = max(lons)
        bbox_coords["lat_min"] = min(lats)
        bbox_coords["lat_max"] = max(lats)

        # Remove old rectangle (if exists)
        if current_rectangle:
            m.remove_layer(current_rectangle)

        # Add new user-selected rectangle
        current_rectangle = Rectangle(
            bounds=[[bbox_coords["lat_min"], bbox_coords["lon_min"]],
                    [bbox_coords["lat_max"], bbox_coords["lon_max"]]],
            color="blue",
            fill_opacity=0.1
        )
        m.add_layer(current_rectangle)

        # Update table with coordinates
        with bbox_output:
            bbox_output.clear_output()
            display(widgets.HTML(
                value=f"""
                <table style="border:1px solid black; border-collapse: collapse;">
                <tr><th style="border:1px solid black; padding:5px">Parameter</th>
                    <th style="border:1px solid black; padding:5px">Value</th></tr>
                <tr><td style="border:1px solid black; padding:5px">lat_min</td><td style="border:1px solid black; padding:5px">{bbox_coords['lat_min']:.4f}</td></tr>
                <tr><td style="border:1px solid black; padding:5px">lat_max</td><td style="border:1px solid black; padding:5px">{bbox_coords['lat_max']:.4f}</td></tr>
                <tr><td style="border:1px solid black; padding:5px">lon_min</td><td style="border:1px solid black; padding:5px">{bbox_coords['lon_min']:.4f}</td></tr>
                <tr><td style="border:1px solid black; padding:5px">lon_max</td><td style="border:1px solid black; padding:5px">{bbox_coords['lon_max']:.4f}</td></tr>
                </table>
                """
            ))

    draw.on_draw(handle_draw)
    m.add_control(draw)

    title = widgets.HTML("<h3>Select Your Area of Interest</h3>")
    hbox = widgets.HBox([m, bbox_output])
    layout = widgets.VBox([title, hbox])
    display(layout)

    return bbox_coords  # Dictionary remains available after drawing

# Usage:
bbox_coords = interactive_bbox_ipy(center=(7,0), zoom=5)


# Step 3: Import IMERG data for the selected region

In [ ]:
print('Selected lat/lon coordinates from the interactive map:')
print(bbox_coords["lat_min"], bbox_coords["lat_max"], bbox_coords["lon_min"], bbox_coords["lon_max"])

def plot_gpm_imerg_box(year, month, day, hour, minute, 
                        lat_min=4, lat_max=16, lon_min=-6, lon_max=3,
                        bbox_coords=None):
    """
    Download and plot GPM IMERG precipitation for a specific time.
    Two figures are created:
    - Left: global (or large region) map
    - Right: zoom on the area defined by bbox_coords or the given limits
    """
    # --- Minute check ---
    if minute not in [0, 30]:
        raise ValueError("IMERG files only exist for hh:00 or hh:30 UTC")
    
    # --- NASA authentication ---
    secrets = netrc()
    username, _, password = secrets.authenticators("urs.earthdata.nasa.gov")
    
    # --- Start and end time ---
    start_time = datetime(year, month, day, hour, minute)
    end_time = start_time + timedelta(minutes=29, seconds=59)
    
    # --- Segment and day of year ---
    base_minutes = hour * 60 + minute
    segment_number = 90 + 30 * (base_minutes // 30 - 3)
    segment_str = f"{segment_number:04d}"
    day_of_year = f"{(start_time - datetime(year,1,1)).days + 1:03d}"
    
    # --- IMERG URL ---
    url = (
        f"https://gpm1.gesdisc.eosdis.nasa.gov/data/GPM_L3/GPM_3IMERGHH.07/"
        f"{year}/{day_of_year}/3B-HHR.MS.MRG.3IMERG."
        f"{start_time.strftime('%Y%m%d')}-S{start_time.strftime('%H%M%S')}"
        f"-E{end_time.strftime('%H%M%S')}.{segment_str}.V07B.HDF5"
    )
    print("Download URL:", url)

    # --- Download and read HDF5 ---
    response = requests.get(url, auth=(username, password))
    response.raise_for_status()
    hdf5_file = h5py.File(io.BytesIO(response.content), 'r')
    
    precip = hdf5_file['Grid/precipitation'][0,:,:]
    lat = hdf5_file['Grid/lat'][:]
    lon = hdf5_file['Grid/lon'][:]
    precip = np.ma.masked_where(precip < 0, precip).T
    
    # --- Select region and BBox ---
    lat_idx = np.where((lat >= lat_min) & (lat <= lat_max))[0]
    lon_idx = np.where((lon >= lon_min) & (lon <= lon_max))[0]
    lat_region = lat[lat_idx]
    lon_region = lon[lon_idx]
    precip_region = precip[np.ix_(lat_idx, lon_idx)]
    lon2d, lat2d = np.meshgrid(lon_region, lat_region)
    
    # --- Discrete colormap ---
    bounds = [0,0.5,2,5,10,15,25,40,100]
    colors = ["none","#add8e6","#0000ff","#00ff00","#ffff00","#ffa500","#ff0000","#ff69b4"]
    cmap = ListedColormap(colors)
    norm = BoundaryNorm(bounds, cmap.N)
    
    # --- Figure with two panels ---
    fig, axs = plt.subplots(1,2, figsize=(16,6),
                            subplot_kw={'projection': ccrs.PlateCarree()})
    
    # Left panel: global region
    pcm1 = axs[0].pcolormesh(lon2d, lat2d, precip_region, cmap=cmap, norm=norm, shading='auto')
    axs[0].coastlines()
    axs[0].add_feature(cfeature.BORDERS)
    axs[0].set_extent([lon_min, lon_max, lat_min, lat_max])
    axs[0].set_title(f"Global Region\n{start_time.strftime('%Y-%m-%d %H:%M UTC')}")
    
    # Right panel: zoom on BBox
    pcm2 = axs[1].pcolormesh(lon2d, lat2d, precip_region, cmap=cmap, norm=norm, shading='auto')
    axs[1].coastlines()
    axs[1].add_feature(cfeature.BORDERS)
    if bbox_coords is not None:
        axs[1].set_extent([bbox_coords["lon_min"], bbox_coords["lon_max"],
                           bbox_coords["lat_min"], bbox_coords["lat_max"]])
    else:
        axs[1].set_extent([lon_min, lon_max, lat_min, lat_max])
    axs[1].set_title("Zoom on BBox")
    
    # Colorbar
    cbar = fig.colorbar(pcm2, ax=axs, orientation='vertical', fraction=0.05, pad=0.02,
                        boundaries=bounds, ticks=bounds[:-1])
    cbar.set_label("Precipitation [mm/h]")
    
    plt.show()


In [ ]:
# Use environment variables for NASA Earthdata credentials
username = os.environ["NASA_USER"]
password = os.environ["NASA_PASS"]

# Plot GPM IMERG precipitation for the selected area
plot_gpm_imerg_box(2023, 10, 10, 0, 0, bbox_coords=bbox_coords)

# Step 4: Visualize multiple time steps for the region of interest

In [ ]:
def plot_gpm_imerg_timeseries(start_time, interval_min, n_steps, bbox_coords):
    """
    Display IMERG images every half hour for a given BBox area.
    
    start_time : datetime
    interval_min : int, interval between images (in minutes)
    n_steps : number of subplots to display
    bbox_coords : dict containing lat_min, lat_max, lon_min, lon_max
    """
    # --- Discrete colormap for precipitation ---
    bounds = [0,0.5,2,5,10,15,25,40,100]
    colors = ["none","#add8e6","#0000ff","#00ff00","#ffff00","#ffa500","#ff0000","#ff69b4"]
    cmap = ListedColormap(colors)
    norm = BoundaryNorm(bounds, cmap.N)
    
    # --- NASA Earthdata authentication ---
    secrets = netrc()
    username, _, password = secrets.authenticators("urs.earthdata.nasa.gov")
    
    # --- Create figure with multiple subplots ---
    fig, axs = plt.subplots(1, n_steps, figsize=(5*n_steps,5),
                            subplot_kw={'projection': ccrs.PlateCarree()})
    
    if n_steps == 1:
        axs = [axs]  # ensure axs is always a list
    
    # --- Loop over each time step ---
    for i in range(n_steps):
        current_time = start_time + timedelta(minutes=i*interval_min)
        hour, minute = current_time.hour, current_time.minute
        
        if minute not in [0,30]:
            raise ValueError("IMERG files exist only for hh:00 or hh:30 UTC")
        
        # --- Compute segment and day of year ---
        base_minutes = hour*60 + minute
        segment_number = 90 + 30*(base_minutes//30 -3)
        segment_str = f"{segment_number:04d}"
        day_of_year = f"{(current_time - datetime(current_time.year,1,1)).days + 1:03d}"
        end_time = current_time + timedelta(minutes=29, seconds=59)
        
        # --- Construct download URL ---
        url = (
            f"https://gpm1.gesdisc.eosdis.nasa.gov/data/GPM_L3/GPM_3IMERGHH.07/"
            f"{current_time.year}/{day_of_year}/3B-HHR.MS.MRG.3IMERG."
            f"{current_time.strftime('%Y%m%d')}-S{current_time.strftime('%H%M%S')}"
            f"-E{end_time.strftime('%H%M%S')}.{segment_str}.V07B.HDF5"
        )
        print("Downloading:", url)
        
        # --- Download and read HDF5 file ---
        response = requests.get(url, auth=(username,password))
        response.raise_for_status()
        hdf5_file = h5py.File(io.BytesIO(response.content), 'r')
        
        precip = hdf5_file['Grid/precipitation'][0,:,:]
        lat = hdf5_file['Grid/lat'][:]
        lon = hdf5_file['Grid/lon'][:]
        precip = np.ma.masked_where(precip<0, precip).T  # mask invalid values
        
        # --- Select BBox area ---
        lat_idx = np.where((lat>=bbox_coords["lat_min"]) & (lat<=bbox_coords["lat_max"]))[0]
        lon_idx = np.where((lon>=bbox_coords["lon_min"]) & (lon<=bbox_coords["lon_max"]))[0]
        lat_region = lat[lat_idx]
        lon_region = lon[lon_idx]
        precip_region = precip[np.ix_(lat_idx, lon_idx)]
        lon2d, lat2d = np.meshgrid(lon_region, lat_region)
        
        # --- Plot the map ---
        pcm = axs[i].pcolormesh(lon2d, lat2d, precip_region, cmap=cmap, norm=norm, shading='auto')
        axs[i].coastlines()
        axs[i].add_feature(cfeature.BORDERS)
        axs[i].set_extent([bbox_coords["lon_min"], bbox_coords["lon_max"],
                           bbox_coords["lat_min"], bbox_coords["lat_max"]])
        axs[i].set_title(current_time.strftime('%Y-%m-%d %H:%M UTC'))
    
    # --- Shared colorbar ---
    cbar = fig.colorbar(pcm, ax=axs, orientation='vertical', fraction=0.05, pad=0.02,
                        boundaries=bounds, ticks=bounds[:-1])
    cbar.set_label("Precipitation [mm/h]")
    plt.show()

In [ ]:
start = datetime(2023,10,10,0,0)
plot_gpm_imerg_timeseries(start_time=start, interval_min=30, n_steps=4, bbox_coords=bbox_coords)

# Step 5: Create an GIF animation of the IMERG data

This script allows you to visualize GPM IMERG precipitation as an animation over a given period and optionally for a defined region (bounding box). The precipitation is displayed using a specific discrete color palette to better distinguish intensities.

Main steps:
1. Authenticate with NASA Earthdata.
2. Generate timestamps based on the chosen interval.
3. Download and read IMERG HDF5 files.
4. Select the area of interest (bounding box) if needed.
5. Create an animation directly in Jupyter Notebook using Matplotlib and Cartopy.
6. Apply a discrete colormap for improved readability.

In [ ]:
def display_imerg_animation_colored(start_time, end_time, interval_min=30, bbox_coords=None):
    """
    Toont een animatie van IMERG-neerslaggegevens
    """

    # NASA Earthdata authenticatie
    secrets = netrc()
    username, _, password = secrets.authenticators("urs.earthdata.nasa.gov")

    # Discrete kleuren definiëren 
    # De bounds geven de neerslagintervallen in mm/h aan
    bounds = [0, 0.5, 2, 5, 10, 15, 25, 40, 100]
    colors = ["none","#add8e6","#0000ff","#00ff00","#ffff00","#ffa500","#ff0000","#ff69b4"]
    cmap = ListedColormap(colors)
    norm = BoundaryNorm(bounds, cmap.N)

    # Genereer timestamps 
    timestamps = []
    t = start_time
    while t <= end_time:
        timestamps.append(t)
        t += timedelta(minutes=interval_min)

    # Download IMERG-gegevens 
    precip_list = []
    lat = lon = None
    for t in tqdm(timestamps, desc="IMERG downloaden"):
        hour, minute = t.hour, t.minute
        if minute not in [0,30]:
            precip_list.append(None)
            continue

        base_minutes = hour*60 + minute
        segment_number = 90 + 30*(base_minutes//30 -3)
        segment_str = f"{segment_number:04d}"
        day_of_year = f"{(t - datetime(t.year,1,1)).days + 1:03d}"
        end_time_seg = t + timedelta(minutes=29, seconds=59)

        url = (
            f"https://gpm1.gesdisc.eosdis.nasa.gov/data/GPM_L3/GPM_3IMERGHH.07/"
            f"{t.year}/{day_of_year}/3B-HHR.MS.MRG.3IMERG."
            f"{t.strftime('%Y%m%d')}-S{t.strftime('%H%M%S')}"
            f"-E{end_time_seg.strftime('%H%M%S')}.{segment_str}.V07B.HDF5"
        )

        try:
            response = requests.get(url, auth=(username,password))
            response.raise_for_status()
            hdf5_file = h5py.File(io.BytesIO(response.content), 'r')

            precip = hdf5_file['Grid/precipitation'][0,:,:]
            lat = hdf5_file['Grid/lat'][:]
            lon = hdf5_file['Grid/lon'][:]
            precip = np.ma.masked_where(precip < 0, precip).T

            if bbox_coords:
                lat_idx = np.where((lat>=bbox_coords["lat_min"]) & (lat<=bbox_coords["lat_max"]))[0]
                lon_idx = np.where((lon>=bbox_coords["lon_min"]) & (lon<=bbox_coords["lon_max"]))[0]
                precip = precip[np.ix_(lat_idx, lon_idx)]
                lat = lat[lat_idx]
                lon = lon[lon_idx]

            precip_list.append(precip)

        except Exception as e:
            print(f"Fout bij {t}: {e}")
            precip_list.append(None)

    # Maak figuur 
    fig, ax = plt.subplots(figsize=(8,6), subplot_kw={'projection': ccrs.PlateCarree()})
    plt.close(fig)
    ax.coastlines()
    ax.add_feature(cfeature.BORDERS)
    if bbox_coords:
        ax.set_extent([bbox_coords["lon_min"], bbox_coords["lon_max"],
                       bbox_coords["lat_min"], bbox_coords["lat_max"]])

    # Begin pcolormesh 
    pcm = ax.pcolormesh(lon, lat, np.zeros_like(precip_list[0]), cmap=cmap, norm=norm, shading='auto')
    cbar = fig.colorbar(pcm, ax=ax, orientation='vertical', ticks=bounds)
    cbar.set_label("Neerslag [mm/h]")

    # Update functie voor animatie 
    def update(frame):
        data = precip_list[frame]
        if data is not None:
            pcm.set_array(data.flatten())
        ax.set_title(f"IMERG: {timestamps[frame].strftime('%Y-%m-%d %H:%M UTC')}")
        return [pcm]

    #  Maak animatie 
    ani = FuncAnimation(fig, update, frames=len(timestamps), interval=600, blit=False)
    return HTML(ani.to_jshtml())

In [ ]:
start_time = datetime(2023,10,10,6,0)
end_time = datetime(2023,10,10,9,0)
bbox_coords = {"lat_min": 4, "lat_max": 16, "lon_min": -6, "lon_max": 3}
display_imerg_animation_colored(start_time, end_time, interval_min=30, bbox_coords=bbox_coords)

## Stap 6: Create a graph displaying the IMERG precipitation data

In [ ]:
def plot_imerg_bars_cumulative_interval(start_time, interval_min, n_steps, bbox_coords):
    """
    Plot a bar chart of average IMERG precipitation over the bbox with cumulative precipitation,
    using automatically generated timestamps.
    """

    # NASA Earthdata authentication
    secrets = netrc()
    username, _, password = secrets.authenticators("urs.earthdata.nasa.gov")

    # Automatically generate timestamps
    imerg_times = [start_time + timedelta(minutes=interval_min*i) for i in range(n_steps)]

    avg_precip_list = []

    # Loop over all IMERG timestamps
    for t in tqdm(imerg_times, desc="IMERG timestamps"):
        # Ensure timestamp matches IMERG files (hh:00 or hh:30)
        hour, minute = t.hour, t.minute
        if minute not in [0,30]:
            avg_precip_list.append(np.nan)
            continue

        # Calculate segment number and day of year
        base_minutes = hour*60 + minute
        segment_number = 90 + 30*(base_minutes//30 -3)
        segment_str = f"{segment_number:04d}"
        day_of_year = f"{(t - datetime(t.year,1,1)).days + 1:03d}"
        end_time = t + timedelta(minutes=29, seconds=59)

        # IMERG file URL
        url = (
            f"https://gpm1.gesdisc.eosdis.nasa.gov/data/GPM_L3/GPM_3IMERGHH.07/"
            f"{t.year}/{day_of_year}/3B-HHR.MS.MRG.3IMERG."
            f"{t.strftime('%Y%m%d')}-S{t.strftime('%H%M%S')}"
            f"-E{end_time.strftime('%H%M%S')}.{segment_str}.V07B.HDF5"
        )

        try:
            # Download and read HDF5 file
            response = requests.get(url, auth=(username,password))
            response.raise_for_status()
            hdf5_file = h5py.File(io.BytesIO(response.content), 'r')

            # Extract data
            precip = hdf5_file['Grid/precipitation'][0,:,:]
            lat = hdf5_file['Grid/lat'][:]
            lon = hdf5_file['Grid/lon'][:]
            precip = np.ma.masked_where(precip < 0, precip).T

            # Select BBox
            lat_idx = np.where((lat>=bbox_coords["lat_min"]) & (lat<=bbox_coords["lat_max"]))[0]
            lon_idx = np.where((lon>=bbox_coords["lon_min"]) & (lon<=bbox_coords["lon_max"]))[0]
            bbox_precip = precip[np.ix_(lat_idx, lon_idx)]

            # Compute average intensity
            avg_precip = float(bbox_precip.mean())
            avg_precip_list.append(avg_precip)

        except Exception as e:
            print(f"Error for {t}: {e}")
            avg_precip_list.append(np.nan)

    # Create DataFrame
    df = pd.DataFrame({
        "timestamp": imerg_times,
        "avg_precip_mmph": avg_precip_list
    })

    # Compute cumulative precipitation
    cum_precip = [0]
    for i in range(1, len(imerg_times)):
        delta_h = (imerg_times[i] - imerg_times[i-1]).total_seconds() / 3600
        cum = cum_precip[-1] + avg_precip_list[i-1] * delta_h
        cum_precip.append(cum)
    df["cum_precip_mm"] = cum_precip

    # Plot
    fig, ax1 = plt.subplots(figsize=(10,5))
    ax1.bar(df["timestamp"], df["avg_precip_mmph"], color='skyblue', 
            label="Average intensity [mm/h]", width=0.02)
    ax1.set_xlabel("Timestamp")
    ax1.set_ylabel("Average precipitation intensity [mm/h]")
    ax1.grid(axis='y', linestyle='--', alpha=0.7)
    ax1.xaxis.set_minor_locator(AutoMinorLocator())

    # Secondary axis for cumulative precipitation
    ax2 = ax1.twinx()
    ax2.plot(df["timestamp"], df["cum_precip_mm"], color='darkblue', label="Cumulative precipitation [mm]")
    ax2.set_ylabel("Cumulative precipitation [mm]")

    # Combined legend
    lines, labels = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax1.legend(lines + lines2, labels + labels2, loc="upper left")

    plt.title(f"IMERG precipitation over the BBox")
    plt.show()

    return df

In [ ]:
start_time = datetime(2023, 10, 10, 0, 0)
interval_min = 120 
n_steps = 3

df = plot_imerg_bars_cumulative_interval(
    start_time=start_time,
    interval_min=interval_min,
    n_steps=n_steps,
    bbox_coords=bbox_coords
)

# Step 7: Determine IMERG precipitation intensity at specific lat-lon locations

In [ ]:
def plot_imerg_points(points, start_time, interval_min, n_steps):
    """
    Retrieves IMERG precipitation for specific points and plots intensity + cumulative.
    """
    # NASA Earthdata authentication 
    secrets = netrc()
    username, _, password = secrets.authenticators("urs.earthdata.nasa.gov")

    # Generate timestamps 
    imerg_times = [start_time + timedelta(minutes=interval_min*i) for i in range(n_steps)]

    # DataFrame to store all values
    df = pd.DataFrame({"timestamp": imerg_times})

    # Loop over each point 
    for point in points:
        lat_p = point["lat"]
        lon_p = point["lon"]
        name_p = point["name"]

        intensity_list = []

        for t in tqdm(imerg_times, desc=f"Fetching IMERG for {name_p}"):
            hour, minute = t.hour, t.minute
            if minute not in [0,30]:
                intensity_list.append(np.nan)
                continue

            base_minutes = hour*60 + minute
            segment_number = 90 + 30*(base_minutes//30 -3)
            segment_str = f"{segment_number:04d}"
            day_of_year = f"{(t - datetime(t.year,1,1)).days + 1:03d}"
            end_time = t + timedelta(minutes=29, seconds=59)

            url = (
                f"https://gpm1.gesdisc.eosdis.nasa.gov/data/GPM_L3/GPM_3IMERGHH.07/"
                f"{t.year}/{day_of_year}/3B-HHR.MS.MRG.3IMERG."
                f"{t.strftime('%Y%m%d')}-S{t.strftime('%H%M%S')}"
                f"-E{end_time.strftime('%H%M%S')}.{segment_str}.V07B.HDF5"
            )

            try:
                response = requests.get(url, auth=(username,password))
                response.raise_for_status()
                hdf5_file = h5py.File(io.BytesIO(response.content), 'r')

                precip = hdf5_file['Grid/precipitation'][0,:,:]
                lat = hdf5_file['Grid/lat'][:]
                lon = hdf5_file['Grid/lon'][:]
                precip = np.ma.masked_where(precip < 0, precip).T

                # Find nearest index to the point
                lat_idx = (np.abs(lat - lat_p)).argmin()
                lon_idx = (np.abs(lon - lon_p)).argmin()
                intensity = float(precip[lat_idx, lon_idx])
                intensity_list.append(intensity)

            except Exception as e:
                print(f"Error for {name_p} at {t}: {e}")
                intensity_list.append(np.nan)

        # Add column for this point
        df[name_p + "_intensity_mmph"] = intensity_list

        # --- Compute cumulative ---
        cum_precip = [0]
        for i in range(1, len(imerg_times)):
            delta_h = (imerg_times[i] - imerg_times[i-1]).total_seconds() / 3600
            cum = cum_precip[-1] + intensity_list[i-1]*delta_h
            cum_precip.append(cum)
        df[name_p + "_cum_mm"] = cum_precip

        # Plot 
        fig, ax1 = plt.subplots(figsize=(10,4))
        ax1.bar(df["timestamp"], df[name_p + "_intensity_mmph"], width=0.02, color='skyblue',
                label="Average intensity [mm/h]")
        ax1.set_xlabel("Timestamp")
        ax1.set_ylabel("Average intensity [mm/h]")
        ax1.grid(axis='y', linestyle='--', alpha=0.7)

        ax2 = ax1.twinx()
        ax2.plot(df["timestamp"], df[name_p + "_cum_mm"], color='darkblue', label="Cumulative precipitation [mm]")
        ax2.set_ylabel("Cumulative precipitation [mm]")

        lines, labels = ax1.get_legend_handles_labels()
        lines2, labels2 = ax2.get_legend_handles_labels()
        ax1.legend(lines + lines2, labels + labels2, loc="upper left")
        plt.title(f"IMERG Precipitation for {name_p}")
        plt.show()

    return df

In [ ]:
# Choose the locations you are interested in
points = [
    {"name": "Lomé", "lat": 6.13, "lon": 1.21},
    {"name": "Ouagadougou", "lat": 12.37, "lon": -1.52},
    {"name": "Nouna", "lat": 12.76, "lon": -3.84},
]

start_time = datetime(2023, 10, 10, 0, 0)           # Choose a start time
interval_min = 30                                   # Select the time interval between IMERG images
n_steps = 3                                         # Select the number of timestamps you want to collect

df_IMERG_precipitation = plot_imerg_points(points, start_time, interval_min, n_steps)
df_IMERG_precipitation

# Save the DataFrame as an Excel file in the same folder as the notebook
# excel_file = "IMERG_points_timeseries.xlsx"
# df_IMERG_precipitation.to_excel(excel_file, index=True)
# print(f"The data has been saved to {excel_file}")

# Congratulations, you have successfully completed the tutorial!